In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [1]:
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline
%config lnlineBackend.figure_format = 'retina'

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import keras

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import RMSprop, Adam
from keras.layers import Conv2D, MaxPooling2D, Flatten   # required for CNN
from keras import backend as K
from keras import losses
from keras import regularizers


from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [2]:
# load the raw data
X_train = np.load('./X_train.npy')
y_train = np.load('./y_train.npy')
print('shape of the training data:', X_train.shape)
print('shape of the target data:', y_train.shape)

shape of the training data: (3313, 150, 150)
shape of the target data: (3313,)


In [3]:
img_rows, img_cols=150,150
x_train, lable_train=X_train[:2650], y_train[:2650]
lable_train=lable_train-1
X_test, y_test=X_train[2650:], y_train[2650:]
y_test-=1
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)


X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
lable_train = keras.utils.to_categorical(lable_train, 4)
y_test = keras.utils.to_categorical(y_test, 4)

In [4]:
noise_salt = np.random.randint(-1, 1, (img_rows, img_cols))
noise_pepper = np.random.randint(-1, 1, (img_rows, img_cols))


In [5]:
noise_salt = noise_salt.reshape(img_rows, img_cols, 1)
noise_pepper = noise_pepper.reshape(img_rows, img_cols, 1)
xnew=np.append(x_train,x_train[:150]+noise_salt+noise_pepper,axis=0)
ynew=np.append(lable_train,lable_train[:150],axis=0)

In [6]:
model = Sequential()
model.add(Conv2D(128,# number of kernels 
                kernel_size=(3, 3),activation='elu',input_shape=(150,150,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))#relu
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))#0.25
model.add(Flatten())
model.add(Dense(128, activation='relu',kernel_regularizer=regularizers.l2(0.02)))
model.add(Dropout(0.25))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(32, activation='relu'))

model.add(Dense(4, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 128)     1280      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 64)        73792     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 32)        18464     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 17, 17, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 17, 17, 32)       

In [ ]:
batch_size = 64
epochs = 25

from keras.callbacks import EarlyStopping
early_stopping_monitor = EarlyStopping(monitor='val_loss', patience=4)


history = model.fit(x_train, lable_train,validation_split=0.5,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          callbacks=[early_stopping_monitor],  
          validation_data=(X_test, y_test))


Train on 2650 samples, validate on 663 samples
Epoch 1/25
2650/2650 [==============================] - 148s 56ms/step - loss: 2.4132 - accuracy: 0.2562 - val_loss: 1.4381 - val_accuracy: 0.3017
Epoch 2/25
 960/2650 [=========>....................] - ETA: 1:43 - loss: 1.4184 - accuracy: 0.2698

In [40]:
score = model.evaluate(X_test, y_test, verbose=1)
print('Test loss: {:.4}'.format(score[0]))
print('Test accuracy: {:.4}'.format(score[1]))

663/663 [==============================] - 0s 461us/step
Test loss: 1.053
Test accuracy: 0.6154


In [ ]:
def transform(result):
    y=[]
    for i in result:
        for j in (np.where(i==np.max(i))):
            for m in j:
                y.append(m+1)
    resultfinal=np.array(y)
    return resultfinal

In [ ]:
X_t = np.load('../input/X_test.npy')
X_t = X_t.reshape(X_t.shape[0], img_rows, img_cols, 1)
resultfinal=transform(model.predict(X_t))
resultfinal[0]

In [ ]:
import pandas as pd
pid=pd.DataFrame(columns=['Id'],data=np.arange(0,829,1))
category=pd.DataFrame(columns=['Category'],data=resultfinal)
testresult=pid.join(category)
testresult

In [ ]:
testresult.to_csv('./result4=0.6712.csv',index=False)